<a href="https://colab.research.google.com/github/HarmanBhutani/ML_projects/blob/main/Assignment4_PartB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install contractions

     |████████████████████████████████| 266kB 34.8MB/s 
     |████████████████████████████████| 327kB 41.6MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85393 sha256=02b041b7738864fa34f8bba51f7ebae24d6405579365e50e064c5d61b267d37c
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [79]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [80]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
import re
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import gensim
from gensim import corpora
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns

In [85]:
df_reut = pd.read_excel('reuters_news_headlines.xlsx', header=None)

In [86]:
df_reut.head()

,0,1,2
0,Headlines,Time,Description
1,TikTok considers London and other locations fo...,Jul 18 2020,TikTok has been in discussions with the UK gov...
2,Disney cuts ad spending on Facebook amid growi...,Jul 18 2020,Walt Disney has become the latest company to ...
3,Trail of missing Wirecard executive leads to B...,Jul 18 2020,Former Wirecard chief operating officer Jan M...
4,Twitter says attackers downloaded data from up...,Jul 18 2020,Twitter Inc said on Saturday that hackers were...


In [87]:
df_reut[0]

0                                                Headlines
1        TikTok considers London and other locations fo...
2        Disney cuts ad spending on Facebook amid growi...
3        Trail of missing Wirecard executive leads to B...
4        Twitter says attackers downloaded data from up...
                               ...                        
32766    Malaysia says never hired British data firm at...
32767    Prosecutors search Volkswagen headquarters in ...
32768     McDonald's sets greenhouse gas reduction targets
32769    Pratt & Whitney to deliver spare A320neo engin...
32770    UK will always consider ways to improve data l...
Name: 0, Length: 32771, dtype: object

In [84]:
df_reut[0] = df_reut[0].apply(lambda x: ' '.join(x))

In [52]:
c = df_reut[0].apply(lambda x: ''.join(x))

In [88]:
nlp = en_core_web_sm.load()

In [89]:
labels = []
for text in df_reut[0]:
    en = nlp(text).ents
    labels.append([l.label_ for l in en])

In [91]:
labels[:15]

[[],
 ['GPE'],
 [],
 ['ORG', 'GPE', 'GPE'],
 ['CARDINAL'],
 ['GPE', 'NORP'],
 ['EVENT', 'WORK_OF_ART'],
 ['CARDINAL'],
 ['GPE', 'ORG'],
 ['ORG'],
 ['GPE'],
 ['ORG'],
 [],
 ['ORG', 'ORG'],
 ['ORG']]

In [92]:
label_bool = []
for idx, lab in enumerate(labels):
    if len(lab)!=0:
        label_bool.append(True)
    else:
        label_bool.append(False)

In [93]:
df_reut['labels'] = labels

In [94]:
df_reut.head()

,0,1,2,labels
0,Headlines,Time,Description,[]
1,TikTok considers London and other locations fo...,Jul 18 2020,TikTok has been in discussions with the UK gov...,[GPE]
2,Disney cuts ad spending on Facebook amid growi...,Jul 18 2020,Walt Disney has become the latest company to ...,[]
3,Trail of missing Wirecard executive leads to B...,Jul 18 2020,Former Wirecard chief operating officer Jan M...,"[ORG, GPE, GPE]"
4,Twitter says attackers downloaded data from up...,Jul 18 2020,Twitter Inc said on Saturday that hackers were...,[CARDINAL]


In [95]:
df_reut[0][9]

'Google bans ads on coronavirus conspiracy theory content'

In [96]:
df_reut = df_reut.loc[label_bool,:]

In [97]:
mlb = MultiLabelBinarizer()

res = pd.DataFrame(mlb.fit_transform(df_reut['labels']),
                   columns=mlb.classes_,
                   index=df_reut.index)

In [98]:
res.head(10)

,CARDINAL,DATE,EVENT,FAC,GPE,LANGUAGE,LAW,LOC,MONEY,NORP,ORDINAL,ORG,PERCENT,PERSON,PRODUCT,QUANTITY,TIME,WORK_OF_ART
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
10,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [99]:
res.sum()

CARDINAL        2656
DATE            4414
EVENT             84
FAC              124
GPE            14093
LANGUAGE          11
LAW               75
LOC              964
MONEY           2670
NORP            3240
ORDINAL          463
ORG            19389
PERCENT          864
PERSON          3965
PRODUCT         1087
QUANTITY          44
TIME              80
WORK_OF_ART       68
dtype: int64

In [100]:
res = res[['ORG','GPE','DATE','CARDINAL']]

In [101]:
res.head()

,ORG,GPE,DATE,CARDINAL
1,0,1,0,0
3,1,1,0,0
4,0,0,0,1
5,0,1,0,0
6,0,0,0,0


In [102]:
from sklearn.model_selection import train_test_split

In [135]:
train, test = train_test_split(df_reut, test_size=0.3)

In [136]:
len(df_reut[0])

30305

In [137]:
len(train)

21213

In [138]:
train

,0,1,2,labels
6477,U.S. consumer watchdog charges Fifth Third Ban...,Mar 09 2020,The U.S. Consumer Financial Protection Bureau ...,"[GPE, ORG]"
13749,Fed's Powell says politics play no role in int...,Sep 06 2019,U.S. Federal Reserve Chair Jerome Powell on Fr...,"[ORG, PERSON]"
12469,Powell: Time for Fed to expand balance sheet '...,Oct 08 2019,Federal Reserve chairman Jerome Powell on Tues...,"[PERSON, ORG]"
19770,Deutsche Bank supervisory board debated merger...,Mar 21 2019,Deutsche Bank's supervisory board debated a po...,"[ORG, TIME]"
4229,Some Accor hotels to admit mild coronavirus ca...,Apr 16 2020,French hotel group Accor said that some of it...,"[PERSON, GPE]"
...,...,...,...,...
23978,Factbox: Amazon to get billions in breaks for ...,Nov 29 2018,(The Factbox updated on Nov. 15 corrects to fi...,"[ORG, CARDINAL]"
20453,GE seeks better insurance returns in junk bond...,Mar 07 2019,General Electric Co will invest more heavily i...,[ORG]
5122,"China factory activity unexpectedly expands, b...",Mar 31 2020,Factory activity in China unexpectedly expande...,[GPE]
1432,Unemployment payments backlog draws hundreds t...,Jun 18 2020,Hundreds of people who lost jobs during the co...,"[CARDINAL, GPE]"


In [139]:
len(test)

9092

In [140]:
test

,0,1,2,labels
30054,BlaBlaCar and AXA launch car insurance product,May 29 2018,"BlaBlaCar, whose amateur chauffeurs share cost...","[CARDINAL, ORG]"
15464,Domino's shares slide as same-store sales disa...,Jul 16 2019,Domino's Pizza Inc reported its slowest U.S. ...,[ORG]
29179,What can Beijing do if China-U.S. trade row wo...,Jun 19 2018,China and the United States sank into a deepen...,"[GPE, GPE, GPE]"
6604,Saudi energy minister says he will 'keep you w...,Mar 06 2020,Saudi Arabia's energy minister said on Friday ...,[NORP]
3243,Exclusive: U.S. airlines burn through $10 bill...,May 05 2020,U.S. airlines are collectively burning more th...,"[GPE, MONEY]"
...,...,...,...,...
359,Exclusive: U.S. turns screws on maritime indus...,Jul 13 2020,Several companies that certify vessels are sea...,"[GPE, GPE]"
19201,Airbus net orders in the red by end of first q...,Apr 04 2019,Airbus suffered more cancellations than orders...,"[ORG, DATE]"
31135,"Refining margins hurt Exxon, Chevron quarterly...",Apr 27 2018,Weak refining margins hurt Exxon Mobil Corp a...,"[ORG, ORG, DATE]"
12545,U.S. unemployment rate hits 3.5%; job growth m...,Oct 04 2019,The U.S. unemployment rate dropped to near a 5...,"[GPE, PERCENT]"


In [141]:
corpus_value = [str (item) for item in df_reut[0]]

In [159]:
corpus_train = [str (item) for item in train[0]]

In [168]:
corpus_test = [str (item) for item in test[0]]

In [162]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,max_features=150,norm='l2', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(corpus_value).toarray()

In [161]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,max_features=150,norm='l2', ngram_range=(1, 2), stop_words='english')
features_train = tfidf.fit_transform(corpus_train).toarray()

In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,max_features=150,norm='l2', ngram_range=(1, 2), stop_words='english')
features_train = tfidf.fit_transform(corpus_train).toarray()

In [169]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,max_features=150,norm='l2', ngram_range=(1, 2), stop_words='english')
features_test = tfidf.fit_transform(corpus_test).toarray()

In [143]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 200, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 51, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [144]:
random_grid

{'bootstrap': [True, False],
 'max_depth': [10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 51, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100,
  105,
  110,
  115,
  121,
  126,
  131,
  136,
  142,
  147,
  152,
  157,
  163,
  168,
  173,
  178,
  184,
  189,
  194,
  200]}

In [163]:
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(features_train, res_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.3min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [164]:
rf_random.best_params_

{'bootstrap': False,
 'max_depth': 42,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 152}

In [165]:
rf = RandomForestClassifier(n_estimators= 152,min_samples_split= 2,min_samples_leaf= 2,max_features= 'auto',max_depth= 42,bootstrap= False)

In [167]:
rf.fit(features_train, res_train)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=42, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=152,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [170]:
rf.predict(features_test)

array([[0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]])

In [148]:
res_train = pd.DataFrame(mlb.fit_transform(train['labels']),
                   columns=mlb.classes_,
                   index=train.index)

In [147]:
train['labels']

6477          [GPE, ORG]
13749      [ORG, PERSON]
12469      [PERSON, ORG]
19770        [ORG, TIME]
4229       [PERSON, GPE]
              ...       
23978    [ORG, CARDINAL]
20453              [ORG]
5122               [GPE]
1432     [CARDINAL, GPE]
22943              [ORG]
Name: labels, Length: 21213, dtype: object

In [155]:
train[0]

6477     U.S. consumer watchdog charges Fifth Third Ban...
13749    Fed's Powell says politics play no role in int...
12469    Powell: Time for Fed to expand balance sheet '...
19770    Deutsche Bank supervisory board debated merger...
4229     Some Accor hotels to admit mild coronavirus ca...
                               ...                        
23978    Factbox: Amazon to get billions in breaks for ...
20453    GE seeks better insurance returns in junk bond...
5122     China factory activity unexpectedly expands, b...
1432     Unemployment payments backlog draws hundreds t...
22943             Dell returns to market with NYSE listing
Name: 0, Length: 21213, dtype: object

In [150]:
res.head(10)

,ORG,GPE,DATE,CARDINAL
1,0,1,0,0
3,1,1,0,0
4,0,0,0,1
5,0,1,0,0
6,0,0,0,0
7,0,0,0,1
8,1,1,0,0
9,1,0,0,0
10,0,1,0,0
11,1,0,0,0


In [153]:
res_train.head(10)

,ORG,GPE,DATE,CARDINAL
6477,1,1,0,0
13749,1,0,0,0
12469,1,0,0,0
19770,1,0,0,0
4229,0,1,0,0
19524,1,0,0,0
9546,0,1,0,0
4480,1,0,0,0
23641,0,1,1,0
1752,1,1,0,0


In [152]:
res_train = res_train[['ORG','GPE','DATE','CARDINAL']]

In [157]:
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(corpus_value, res)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    1.6s finished


ValueError: ignored